## Install packages

In [30]:
#%pip install -U pandas numpy scikit-learn scipy matplotlib tqdm nltk
#%pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

## Imports + config + device

In [31]:
import os, re, random, time
from dataclasses import dataclass
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

print("Torch:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Torch: 2.5.1+cu121
Device: cuda
GPU: NVIDIA GeForce RTX 2050


## Reproducibility + global settings

In [32]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

@dataclass
class CFG:
    seed: int = 42
    data_path: str = "./data/jigsaw/train.csv"
    subsample_n: int | None = 200_000  # set None to use full file (big)
    test_size: float = 0.1
    val_size: float = 0.1  # fraction of remaining train for validation
    label_threshold: float = 0.5

    # TF-IDF
    max_features: int = 50_000
    ngram_range: tuple = (1, 2)
    min_df: int = 2

    # PyTorch training
    batch_size: int = 256
    epochs: int = 3
    lr: float = 1e-3
    weight_decay: float = 1e-4
    optimizer: str = "adam"  # "adam" or "sgd"
    momentum: float = 0.9
    grad_clip_norm: float | None = 1.0

    # Sequence model
    seq_max_len: int = 200
    vocab_size: int = 50_000
    emb_dim: int = 128
    lstm_hidden: int = 128
    lstm_layers: int = 1
    dropout: float = 0.3

cfg = CFG()
seed_everything(cfg.seed)
cfg


CFG(seed=42, data_path='./data/jigsaw/train.csv', subsample_n=200000, test_size=0.1, val_size=0.1, label_threshold=0.5, max_features=50000, ngram_range=(1, 2), min_df=2, batch_size=256, epochs=3, lr=0.001, weight_decay=0.0001, optimizer='adam', momentum=0.9, grad_clip_norm=1.0, seq_max_len=200, vocab_size=50000, emb_dim=128, lstm_hidden=128, lstm_layers=1, dropout=0.3)

## Load Jigsaw train.csv + label + subsample

In [33]:
df = pd.read_csv(cfg.data_path)
print(df.shape)
df = df.dropna(subset=["comment_text", "target"]).reset_index(drop=True)

df["label"] = (df["target"] >= cfg.label_threshold).astype(int)

# Optional: subsample (keep label distribution)
if cfg.subsample_n is not None and len(df) > cfg.subsample_n:
    df, _ = train_test_split(
        df,
        train_size=cfg.subsample_n,
        stratify=df["label"],
        random_state=cfg.seed
    )
    df = df.reset_index(drop=True)

df[["target", "label"]].describe()


(1804874, 45)


,target,label
count,200000.000000,200000.000000
mean,0.103228,0.079970
std,0.197294,0.271247
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.166667,0.000000
max,1.000000,1.000000


## Train/Val/Test split

In [34]:
X = df["comment_text"].astype(str).values
y = df["label"].values

# First: split out test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y,
    test_size=cfg.test_size,
    stratify=y,
    random_state=cfg.seed
)

# Then: split train/val from remaining
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=cfg.val_size,
    stratify=y_trainval,
    random_state=cfg.seed
)

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))
print("Pos rate train/val/test:", y_train.mean(), y_val.mean(), y_test.mean())


Train: 162000 Val: 18000 Test: 20000
Pos rate train/val/test: 0.07997530864197532 0.07994444444444444 0.07995


## Preprocessing: tokenization + stemming + lemmatization

In [35]:
# NLTK downloads (run once)
nltk.download("wordnet")
nltk.download("omw-1.4")

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

_token_re = re.compile(r"[A-Za-z']+")

def basic_tokenize(text: str):
    return _token_re.findall(text.lower())

def stem_text(text: str) -> str:
    toks = basic_tokenize(text)
    toks = [stemmer.stem(t) for t in toks]
    return " ".join(toks)

def lemma_text(text: str) -> str:
    toks = basic_tokenize(text)
    toks = [lemmatizer.lemmatize(t) for t in toks]
    return " ".join(toks)

# Choose one:
PREPROC_MODE = "raw"  # "raw" | "stem" | "lemma"

def preprocess_array(arr):
    if PREPROC_MODE == "raw":
        return arr
    fn = stem_text if PREPROC_MODE == "stem" else lemma_text
    return np.array([fn(x) for x in tqdm(arr, desc=f"preprocess={PREPROC_MODE}")], dtype=object)

X_train_p = preprocess_array(X_train)
X_val_p   = preprocess_array(X_val)
X_test_p  = preprocess_array(X_test)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stjepan.vinski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\stjepan.vinski\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Common evaluation helper

In [36]:
def eval_binary(y_true, y_pred, name="model"):
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    print(f"{name}: acc={acc:.4f} precision={p:.4f} recall={r:.4f} f1={f1:.4f}")
    return {"acc": acc, "precision": p, "recall": r, "f1": f1}


## TF-IDF + Logistic Regression (baseline + cross-validation)

### Vectorize TF-IDF

In [37]:
tfidf = TfidfVectorizer(
    max_features=cfg.max_features,
    ngram_range=cfg.ngram_range,
    min_df=cfg.min_df
)

Xtr_tfidf = tfidf.fit_transform(X_train_p)
Xva_tfidf = tfidf.transform(X_val_p)
Xte_tfidf = tfidf.transform(X_test_p)

Xtr_tfidf.shape, Xva_tfidf.shape


((162000, 50000), (18000, 50000))

### Logistic Regression baseline

In [38]:
logreg = LogisticRegression(
    max_iter=200,
    n_jobs=-1,
    class_weight="balanced"  # useful for skew
)

logreg.fit(Xtr_tfidf, y_train)

val_pred = logreg.predict(Xva_tfidf)
test_pred = logreg.predict(Xte_tfidf)

metrics_logreg_val = eval_binary(y_val, val_pred, "TFIDF+LogReg (val)")
metrics_logreg_test = eval_binary(y_test, test_pred, "TFIDF+LogReg (test)")


c:\Stjepan\Fax\NepredniML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


TFIDF+LogReg (val): acc=0.9043 precision=0.4405 recall=0.7311 f1=0.5498
TFIDF+LogReg (test): acc=0.9038 precision=0.4401 recall=0.7486 f1=0.5543


### Cross-validation

In [39]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfg.seed)
cv_scores = cross_val_score(logreg, tfidf.transform(X_trainval), y_trainval, cv=skf, scoring="f1")
print("CV F1 scores:", cv_scores)
print("CV F1 mean/std:", cv_scores.mean(), cv_scores.std())


c:\Stjepan\Fax\NepredniML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
c:\Stjepan\Fax\NepredniML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
c:\Stjepan\Fax\NepredniML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
c:\Stjepan\Fax\NepredniML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please

CV F1 scores: [0.53827288 0.55464664 0.54797413 0.54752148 0.54862385]
CV F1 mean/std: 0.5474077973040654 0.005247094867706876


## TF-IDF + MLP

### Sparse TF-IDF

In [40]:
class SparseTfidfDataset(Dataset):
    def __init__(self, X_csr, y):
        self.X = X_csr.tocsr()
        self.y = y.astype(np.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        row = self.X[idx]
        x = torch.from_numpy(row.toarray().ravel()).float()
        y = torch.tensor(self.y[idx]).float()
        return x, y

train_ds = SparseTfidfDataset(Xtr_tfidf, y_train)
val_ds   = SparseTfidfDataset(Xva_tfidf, y_val)
test_ds  = SparseTfidfDataset(Xte_tfidf, y_test)

train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False)

Xtr_tfidf.shape


(162000, 50000)

### MLP model

In [41]:
class TfidfMLP(nn.Module):
    def __init__(self, in_dim, hidden=256, dropout=0.3, activation="relu"):
        super().__init__()
        self.activation = activation.lower()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.bn1 = nn.BatchNorm1d(hidden)
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden, 1)

        # weight init
        if self.activation == "relu":
            nn.init.kaiming_normal_(self.fc1.weight)
        else:
            nn.init.xavier_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x):
        x = self.bn1(self.fc1(x))
        if self.activation == "relu":
            x = F.relu(x)
        elif self.activation == "tanh":
            x = torch.tanh(x)
        elif self.activation == "sigmoid":
            x = torch.sigmoid(x)
        else:
            raise ValueError("activation must be relu/tanh/sigmoid")

        x = self.drop(x)
        return self.fc2(x).squeeze(1)  # logits


### Train/eval

In [42]:
def make_optimizer(model):
    if cfg.optimizer.lower() == "adam":
        return torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    if cfg.optimizer.lower() == "sgd":
        return torch.optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)
    raise ValueError("optimizer must be adam/sgd")

@torch.no_grad()
def eval_loader(model, loader):
    model.eval()
    ys, ps = [], []
    for x, y in loader:
        x = x.to(device)
        logits = model(x)
        prob = torch.sigmoid(logits).cpu().numpy()
        ys.append(y.numpy())
        ps.append(prob)
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(int)
    return y_true.astype(int), y_prob, y_pred

def fit_binary(model, train_loader, val_loader, epochs=3):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    opt = make_optimizer(model)

    best_f1 = -1
    best_state = None

    for ep in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        n = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            opt.zero_grad(set_to_none=True)
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()

            if cfg.grad_clip_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip_norm)

            opt.step()

            bs = x.size(0)
            total_loss += loss.item() * bs
            n += bs

        yv, pv, yv_pred = eval_loader(model, val_loader)
        m = eval_binary(yv, yv_pred, f"MLP val ep{ep}")
        print(f"Epoch {ep}/{epochs} train_loss={total_loss/n:.4f}")

        if m["f1"] > best_f1:
            best_f1 = m["f1"]
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    if best_state is not None:
        model.load_state_dict(best_state)

    return model


### Train the MLP

In [43]:
cfg.optimizer = "adam"
cfg.lr = 1e-3
cfg.epochs = 3

mlp = TfidfMLP(in_dim=Xtr_tfidf.shape[1], hidden=256, dropout=0.3, activation="relu")
mlp = fit_binary(mlp, train_loader, val_loader, epochs=cfg.epochs)

yt, pt, ypred = eval_loader(mlp, test_loader)
metrics_mlp_test = eval_binary(yt, ypred, "TFIDF+MLP (test)")


MLP val ep1: acc=0.9403 precision=0.7593 recall=0.3704 f1=0.4979
Epoch 1/3 train_loss=0.2189
MLP val ep2: acc=0.9379 precision=0.6882 recall=0.4079 f1=0.5122
Epoch 2/3 train_loss=0.1179
MLP val ep3: acc=0.9358 precision=0.6336 recall=0.4663 f1=0.5372
Epoch 3/3 train_loss=0.0756
TFIDF+MLP (test): acc=0.9356 precision=0.6312 recall=0.4678 f1=0.5374


### Activation comparison

In [44]:
for act in ["relu", "tanh", "sigmoid"]:
    m = TfidfMLP(in_dim=Xtr_tfidf.shape[1], hidden=256, dropout=0.3, activation=act)
    m = fit_binary(m, train_loader, val_loader, epochs=2)
    yt, pt, ypred = eval_loader(m, test_loader)
    eval_binary(yt, ypred, f"TFIDF+MLP act={act} (test)")


MLP val ep1: acc=0.9387 precision=0.7417 recall=0.3572 f1=0.4822
Epoch 1/2 train_loss=0.2199
MLP val ep2: acc=0.9387 precision=0.6995 recall=0.4093 f1=0.5164
Epoch 2/2 train_loss=0.1172
TFIDF+MLP act=relu (test): acc=0.9392 precision=0.7009 recall=0.4178 f1=0.5235
MLP val ep1: acc=0.9396 precision=0.7230 recall=0.3954 f1=0.5112
Epoch 1/2 train_loss=0.3069
MLP val ep2: acc=0.9384 precision=0.6901 recall=0.4163 f1=0.5193
Epoch 2/2 train_loss=0.1330
TFIDF+MLP act=tanh (test): acc=0.9385 precision=0.6855 recall=0.4253 f1=0.5249
MLP val ep1: acc=0.9397 precision=0.7736 recall=0.3468 f1=0.4789
Epoch 1/2 train_loss=0.2145
MLP val ep2: acc=0.9411 precision=0.7342 recall=0.4128 f1=0.5285
Epoch 2/2 train_loss=0.1482
TFIDF+MLP act=sigmoid (test): acc=0.9403 precision=0.7170 recall=0.4184 f1=0.5284


### Compare SGD vs Adam

In [45]:
cfg.optimizer = "sgd"
cfg.lr = 0.05
cfg.momentum = 0.9

m = TfidfMLP(in_dim=Xtr_tfidf.shape[1], hidden=256, dropout=0.3, activation="relu")
m = fit_binary(m, train_loader, val_loader, epochs=2)
yt, pt, ypred = eval_loader(m, test_loader)
eval_binary(yt, ypred, "TFIDF+MLP SGD (test)")


MLP val ep1: acc=0.9420 precision=0.7411 recall=0.4218 f1=0.5376
Epoch 1/2 train_loss=0.1893
MLP val ep2: acc=0.9361 precision=0.6289 recall=0.4899 f1=0.5508
Epoch 2/2 train_loss=0.1008
TFIDF+MLP SGD (test): acc=0.9372 precision=0.6324 recall=0.5122 f1=0.5660


{'acc': 0.9372,
 'precision': 0.6324324324324324,
 'recall': 0.5121951219512195,
 'f1': 0.5659986178299931}

## LSTM and BiLSTM+Attention

### Build vocabulary + encode sequences

In [46]:
PAD = 0
UNK = 1

def build_vocab(texts, vocab_size=50_000):
    counter = Counter()
    for t in tqdm(texts, desc="build_vocab"):
        counter.update(basic_tokenize(t))
    most = counter.most_common(vocab_size - 2)
    stoi = {"<PAD>": PAD, "<UNK>": UNK}
    for i, (w, _) in enumerate(most, start=2):
        stoi[w] = i
    return stoi

stoi = build_vocab(X_train_p, vocab_size=cfg.vocab_size)
itos_size = len(stoi)
itos_size


build_vocab: 100%|██████████| 162000/162000 [00:02<00:00, 70844.53it/s]


50000

In [47]:
def encode_text(text, stoi, max_len=200):
    toks = basic_tokenize(text)
    ids = [stoi.get(w, UNK) for w in toks[:max_len]]
    if len(ids) == 0:
        ids = [UNK]
    return ids


def pad_batch(batch_ids, pad_id=PAD):
    lens = torch.tensor([max(1, len(x)) for x in batch_ids], dtype=torch.long)
    maxlen = int(lens.max().item())
    padded = torch.full((len(batch_ids), maxlen), pad_id, dtype=torch.long)

    for i, ids in enumerate(batch_ids):
        if len(ids) == 0:
            ids = [UNK]
        padded[i, :len(ids)] = torch.tensor(ids, dtype=torch.long)

    return padded, lens



### Sequence Dataset + collate_fn

In [48]:
class SeqDataset(Dataset):
    def __init__(self, texts, labels, stoi, max_len):
        self.texts = texts
        self.labels = labels.astype(np.float32)
        self.stoi = stoi
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        ids = encode_text(self.texts[idx], self.stoi, self.max_len)
        y = self.labels[idx]
        return ids, y

def collate_fn(batch):
    ids_list, ys = zip(*batch)
    x, lens = pad_batch(ids_list, pad_id=PAD)
    y = torch.tensor(ys, dtype=torch.float32)
    return x, lens, y

seq_train = SeqDataset(X_train_p, y_train, stoi, cfg.seq_max_len)
seq_val   = SeqDataset(X_val_p, y_val, stoi, cfg.seq_max_len)
seq_test  = SeqDataset(X_test_p, y_test, stoi, cfg.seq_max_len)

seq_train_loader = DataLoader(seq_train, batch_size=cfg.batch_size, shuffle=True, collate_fn=collate_fn)
seq_val_loader   = DataLoader(seq_val, batch_size=cfg.batch_size, shuffle=False, collate_fn=collate_fn)
seq_test_loader  = DataLoader(seq_test, batch_size=cfg.batch_size, shuffle=False, collate_fn=collate_fn)


### LSTM classifier

In [49]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden=128, layers=1, dropout=0.3, bidir=False):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
        self.lstm = nn.LSTM(
            emb_dim, hidden, num_layers=layers,
            batch_first=True, dropout=dropout if layers > 1 else 0.0,
            bidirectional=bidir
        )
        out_dim = hidden * (2 if bidir else 1)
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(out_dim, 1)

        nn.init.xavier_normal_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, lens):
        emb = self.emb(x)
        packed = nn.utils.rnn.pack_padded_sequence(emb, lens.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (h, c) = self.lstm(packed)

        # last layer hidden
        if self.lstm.bidirectional:
            h_last = torch.cat([h[-2], h[-1]], dim=1)
        else:
            h_last = h[-1]

        h_last = self.drop(h_last)
        logits = self.fc(h_last).squeeze(1)
        return logits


### Train/eval loops for sequence models

In [50]:
@torch.no_grad()
def eval_seq(model, loader):
    model.eval()
    ys, ps = [], []
    for x, lens, y in loader:
        x, lens = x.to(device), lens.to(device)
        logits = model(x, lens)
        prob = torch.sigmoid(logits).cpu().numpy()
        ys.append(y.numpy())
        ps.append(prob)
    y_true = np.concatenate(ys).astype(int)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(int)
    return y_true, y_prob, y_pred

def fit_seq(model, train_loader, val_loader, epochs=3):
    model = model.to(device)
    crit = nn.BCEWithLogitsLoss()
    opt = make_optimizer(model)

    best_f1 = -1
    best_state = None

    for ep in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        n = 0

        for x, lens, y in train_loader:
            x, lens, y = x.to(device), lens.to(device), y.to(device)
            opt.zero_grad(set_to_none=True)
            logits = model(x, lens)
            loss = crit(logits, y)
            loss.backward()

            if cfg.grad_clip_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip_norm)

            opt.step()

            bs = x.size(0)
            total_loss += loss.item() * bs
            n += bs

        yv, pv, yv_pred = eval_seq(model, val_loader)
        m = eval_binary(yv, yv_pred, f"SEQ val ep{ep}")
        print(f"Epoch {ep}/{epochs} train_loss={total_loss/n:.4f}")

        if m["f1"] > best_f1:
            best_f1 = m["f1"]
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    if best_state is not None:
        model.load_state_dict(best_state)

    return model


### Train LSTM

In [51]:
cfg.optimizer = "adam"
cfg.lr = 1e-3
cfg.epochs = 3
cfg.grad_clip_norm = 1.0  # very relevant for RNN/LSTM

lstm = LSTMClassifier(vocab_size=len(stoi), emb_dim=cfg.emb_dim, hidden=cfg.lstm_hidden,
                      layers=cfg.lstm_layers, dropout=cfg.dropout, bidir=False)
lstm = fit_seq(lstm, seq_train_loader, seq_val_loader, epochs=cfg.epochs)

yt, pt, ypred = eval_seq(lstm, seq_test_loader)
metrics_lstm_test = eval_binary(yt, ypred, "LSTM (test)")


SEQ val ep1: acc=0.9253 precision=0.7714 recall=0.0938 f1=0.1673
Epoch 1/3 train_loss=0.2748
SEQ val ep2: acc=0.9366 precision=0.6472 recall=0.4538 f1=0.5335
Epoch 2/3 train_loss=0.2050
SEQ val ep3: acc=0.9420 precision=0.7778 recall=0.3843 f1=0.5144
Epoch 3/3 train_loss=0.1709
LSTM (test): acc=0.9356 precision=0.6402 recall=0.4440 f1=0.5244


## BiLSTM + Attention

### Additive attention on token outputs

In [52]:
class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden=128, layers=1, dropout=0.3):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD)
        self.lstm = nn.LSTM(
            emb_dim, hidden, num_layers=layers,
            batch_first=True, dropout=dropout if layers > 1 else 0.0,
            bidirectional=True
        )
        self.attn = nn.Linear(hidden * 2, 1)  # score each timestep
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden * 2, 1)

        nn.init.xavier_normal_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, lens):
        emb = self.emb(x)

        packed = nn.utils.rnn.pack_padded_sequence(emb, lens.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)  # (B,T,H*2)

        # mask padding
        B, T, _ = out.shape
        mask = torch.arange(T, device=lens.device).unsqueeze(0) >= lens.unsqueeze(1)  # (B,T) True for pad

        scores = self.attn(out).squeeze(-1)  # (B,T)
        scores = scores.masked_fill(mask, -1e9)
        weights = torch.softmax(scores, dim=1)  # (B,T)

        context = torch.bmm(weights.unsqueeze(1), out).squeeze(1)  # (B,H*2)
        context = self.drop(context)
        logits = self.fc(context).squeeze(1)
        return logits


### Train attention model

In [53]:
attn_model = BiLSTMAttention(vocab_size=len(stoi), emb_dim=cfg.emb_dim, hidden=cfg.lstm_hidden,
                            layers=cfg.lstm_layers, dropout=cfg.dropout)
attn_model = fit_seq(attn_model, seq_train_loader, seq_val_loader, epochs=cfg.epochs)

yt, pt, ypred = eval_seq(attn_model, seq_test_loader)
metrics_attn_test = eval_binary(yt, ypred, "BiLSTM+Attention (test)")


SEQ val ep1: acc=0.9387 precision=0.7530 recall=0.3475 f1=0.4755
Epoch 1/3 train_loss=0.2407
SEQ val ep2: acc=0.9442 precision=0.7686 recall=0.4315 f1=0.5527
Epoch 2/3 train_loss=0.1817
SEQ val ep3: acc=0.9452 precision=0.7475 recall=0.4753 f1=0.5811
Epoch 3/3 train_loss=0.1637
BiLSTM+Attention (test): acc=0.9444 precision=0.7387 recall=0.4703 f1=0.5747


## Autoencoder (MSE) on TF-IDF + classifier

### Autoencoder on TF-IDF vectors

In [54]:
class TfidfAutoencoder(nn.Module):
    def __init__(self, in_dim, bottleneck=256):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(in_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, bottleneck),
            nn.ReLU()
        )
        self.dec = nn.Sequential(
            nn.Linear(bottleneck, 1024),
            nn.ReLU(),
            nn.Linear(1024, in_dim)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        z = self.enc(x)
        xhat = self.dec(z)
        return z, xhat


In [55]:
# Use smaller TF-IDF for AE to keep RAM reasonable
tfidf_ae = TfidfVectorizer(max_features=20_000, ngram_range=(1,2), min_df=2)
Xtr_ae = tfidf_ae.fit_transform(X_train_p)
Xva_ae = tfidf_ae.transform(X_val_p)
Xte_ae = tfidf_ae.transform(X_test_p)

ae_train = SparseTfidfDataset(Xtr_ae, y_train)
ae_val   = SparseTfidfDataset(Xva_ae, y_val)
ae_test  = SparseTfidfDataset(Xte_ae, y_test)

ae_train_loader = DataLoader(ae_train, batch_size=256, shuffle=True)
ae_val_loader   = DataLoader(ae_val, batch_size=256, shuffle=False)
ae_test_loader  = DataLoader(ae_test, batch_size=256, shuffle=False)

in_dim = Xtr_ae.shape[1]


In [56]:
def train_autoencoder(ae, loader, epochs=3):
    ae = ae.to(device)
    opt = torch.optim.Adam(ae.parameters(), lr=1e-3, weight_decay=1e-5)
    loss_fn = nn.MSELoss()

    for ep in range(1, epochs+1):
        ae.train()
        total = 0.0
        n = 0
        for x, _ in loader:
            x = x.to(device)
            opt.zero_grad(set_to_none=True)
            z, xhat = ae(x)
            loss = loss_fn(xhat, x)
            loss.backward()
            opt.step()
            total += loss.item() * x.size(0)
            n += x.size(0)
        print(f"AE epoch {ep}/{epochs} mse={total/n:.6f}")

    return ae

ae = TfidfAutoencoder(in_dim=in_dim, bottleneck=256)
ae = train_autoencoder(ae, ae_train_loader, epochs=3)


AE epoch 1/3 mse=0.000049
AE epoch 2/3 mse=0.000049
AE epoch 3/3 mse=0.000049


### Encode TF-IDF through AE and train a classifier on embeddings

In [57]:
@torch.no_grad()
def encode_dataset(ae, loader):
    ae.eval()
    Z, Y = [], []
    for x, y in loader:
        x = x.to(device)
        z, _ = ae(x)
        Z.append(z.cpu().numpy())
        Y.append(y.numpy())
    return np.vstack(Z), np.concatenate(Y).astype(int)

Ztr, Ytr = encode_dataset(ae, ae_train_loader)
Zva, Yva = encode_dataset(ae, ae_val_loader)
Zte, Yte = encode_dataset(ae, ae_test_loader)

clf = LogisticRegression(max_iter=300, class_weight="balanced")
clf.fit(Ztr, Ytr)

pred = clf.predict(Zte)
metrics_ae_test = eval_binary(Yte, pred, "AE-Embeddings + LogReg (test)")


AE-Embeddings + LogReg (test): acc=0.9201 precision=0.0000 recall=0.0000 f1=0.0000


## Final comparison table - all models

In [58]:
rows = []
def add(name, metrics):
    rows.append({"model": name, **metrics})

add("TFIDF+LogReg", metrics_logreg_test)
add("TFIDF+MLP", metrics_mlp_test)
add("LSTM", metrics_lstm_test)
add("BiLSTM+Attention", metrics_attn_test)
add("AE+LogReg", metrics_ae_test)

pd.DataFrame(rows).sort_values("f1", ascending=False)


,model,acc,precision,recall,f1
3,BiLSTM+Attention,0.94435,0.738703,0.470294,0.574704
0,TFIDF+LogReg,0.90375,0.440074,0.748593,0.554295
1,TFIDF+MLP,0.93560,0.631224,0.467792,0.537356
2,LSTM,0.93560,0.640216,0.444028,0.524372
4,AE+LogReg,0.92005,0.000000,0.000000,0.000000
